In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import pickle
import json
import matplotlib.pyplot as plt
from init_objects import *
from functions.stylizedfacts import *
from functions.helpers import organise_data
from functions.inequality import gini, lorenz_curve
from distribution_model import *
import statsmodels.api as sm
from SALib.sample import latin
from matplotlib import style
from collections import OrderedDict
from functions.indirect_calibration import distr_model_performance
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy
from functions.find_bubbles import *
from statsmodels.iolib.summary2 import summary_col

# Asset boom-busts and wealth inequality


## Are asset boom-busts episodes associated with increased wealth inequality?
To find out whether asset boom-busts are associated with increased inequality, I compare the difference in wealth inequality between periods of boom-bust to reference periods such an episode did not occur. 

In [ ]:
# simulate --> recognize boom-bust, compare to equal lenght periods with no bubble. 